In [10]:
using DynamicPolynomials, SparseArrays

println("***Problem setting***")

n=5

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

function generate_random_poly(v)
    lv=length(v)
    c=2*rand(Float64,lv).-1
    return sum(c[j]*v[j] for j in 1:lv)
end
# random quadratic objective function f
v=reverse(monomials(x,2))
f=generate_random_poly(v)


# unit sphere constraint
g=[1.0-sum(x.^2)] #type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")

l=ceil(Int64, n/4)

h=Vector{Polynomial{true,Float64}}(undef,l)
randx=2*rand(n).-1# create a feasible solution
randx=rand(1)[1]*randx./sqrt(sum(randx.^2))


for j in 1:l
    h[j]=generate_random_poly(v[2:end])
    h[j]-=h[j](x => randx) #make constraints feasible
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=5
Number of inequality constraints: m=1
Number of equality constraints: l=2


In [11]:
include("../src/ctpPOP.jl")
using .ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=false);

In [12]:
k=2

println("Relaxed order: k=",k)
println("====================")
t=1
println("Sparse order: t=",t)

Relaxed order: k=2
Sparse order: t=1


In [13]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_TS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=4


  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=114
Modeling time:
  0.404686 seconds (569.57 k allocations: 30.066 MiB)
---------------
| Parameters: |
---------------
n:       114
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.75221852D+00  WK= 0.4444D+00  QK= 0.2222D+00
 NIT=    2  NFE=    2  F= 0.32616106D+00  WK= 0.4717D+00  QK= 0.1644D+00
 NIT=    3  NFE=    4  F= 0.31719502D+00  WK= 0.7797D-01  QK= 0.6933D-01
 NIT=    4  NFE=    5  F= 0.24006084D+00  WK= 0.6067D+01  QK= 0.6591D-01
 NIT=    5  NFE=    6  F= 0.24006084D+00  WK= 0.3300D+00  QK= 0.1814D+00
 NIT=    6  NFE=   51  F= 0.24006084D+00  WK= 0.3602D-01  QK= 0.6591D-01
 NIT=    7  NFE=   56  F= 0.23874825D+00  WK= 0.7051D-02  QK= 0.1813D+00
 NIT=    8  NFE=   57  F= 0.23874825D+00  WK= 0.3625D+00  QK= 0.1813D+00
 NIT=    9  NFE=   58  F= 0.23874825D+00  WK= 0.2227D+00  QK= 0.1129D+00
 NIT=   10  NFE=   59  F= 0.23834515D+00

-0.8742672380716684

In [14]:
using TSSOS

@time begin
    ~,~,data=tssos_first([[f];g;h],x,k,numeq=length(h),TS="block",quotient=false);
    for j in 1:t-1
        ~,~,data=tssos_higher!(data,TS="block")
    end
end

------------------------------------------------------
The sizes of blocks:
[16, 5]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 86              
  Cones                  : 0               
  Scalar variables       : 34              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  

In [15]:
include("../src/ctpPOP.jl")
using .ctpPOP

opt_val=ctpPOP.POP_TS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,t,
                         tol=1e-3,use_eqcons_to_get_constant_trace=false)

  Computing constant trace status: OPTIMAL
  Constant trace: ak = 3.0
  Number of blocks: omega=4


  Size of the largest block: s^max=16
  Number of equality trace constraints: zeta=114
Modeling time:
  0.345734 seconds (569.58 k allocations: 29.678 MiB)
iter=1   val=-4.104586086277793   gap=0.0   feas=3.76893665109561
iter=2   val=-1.3681953620925977   gap=0.39459333571988336   feas=1.04212221683159
iter=4   val=-1.00951524521898   gap=0.3023897228268982   feas=0.6660302774770013
iter=8   val=-1.317285105875222   gap=0.010242647720584563   feas=0.6364093669150612
iter=16   val=-0.8495434365499714   gap=0.18437741048106435   feas=0.14252354380999588
iter=32   val=-0.8343627529440077   gap=0.062019634784428644   feas=0.13188176975859864
iter=64   val=-0.8662251933156929   gap=0.07962574360419919   feas=0.010376277257840233
iter=128   val=-0.8572024848158895   gap=0.03135839737119633   feas=0.023365476809104148
iter=256   val=-0.8731864263695996   gap=0.06382728699792496   feas=0.002243515196766036
iter=512   val=-0.8700515078768231   gap=0.023367803448634535   feas=0.0048280829319212

-0.8728071344446223

In [16]:
include("../src/ctpPOP.jl")
using .ctpPOP

optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       48
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:       48

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

0.0